# Example of binary Classification

This notebook is a modified and simplified version of the [original notebook](https://github.com/ageron/handson-ml/blob/master/03_classification.ipynb) by Aurelien Geron in his book

# Setup

First, let's make sure this notebook works well in both python 2 and 3, import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures:

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty inline figures. this is a different magic commands than %matplotlib notebook
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/tmp/"
CHAPTER_ID = "classification"

def save_fig(fig_id, tight_layout=True):
    #path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    path = os.path.join(PROJECT_ROOT_DIR, fig_id + ".png")
    print("Saving figure", path)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# MNIST Hanwritten database

More information about [MNIST database](http://yann.lecun.com/exdb/mnist/)

**Warning**: `fetch_mldata()` is deprecated since Scikit-Learn 0.20. You should use `fetch_openml()` instead. However, it returns the unsorted MNIST dataset, whereas `fetch_mldata()` returned the dataset sorted by target (the training set and the test test were sorted separately). In general, this is fine, but if you want to get the exact same results as before, you need to sort the dataset using the following function:

In [ ]:
def sort_by_target(mnist):
    reorder_train = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[:60000])]))[:, 1]
    reorder_test = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[60000:])]))[:, 1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000] = mnist.target[reorder_train]
    mnist.data[60000:] = mnist.data[reorder_test + 60000]
    mnist.target[60000:] = mnist.target[reorder_test + 60000]

In [ ]:
try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, cache=True)
    mnist.target = mnist.target.astype(np.int8) # fetch_openml() returns targets as strings
    sort_by_target(mnist) # fetch_openml() returns an unsorted dataset
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')

In [ ]:
mnist

There are 70000 entries (rows) each with 784 columns

In [ ]:
mnist.data.shape

Each row is an image of 784 pixels. 


In [ ]:
mnist.data[0,:]

For each __image__ (called data point) we have a __target__ which is the correct identification of the digit in the image

In [ ]:
mnist.target.shape

In [ ]:
mnist.target[42340]

In [ ]:
mnist["data"], mnist["target"]

`mnist.data` and `mnist["data"]` are the same thing

In [ ]:
mnist["data"][0,:]

each line has data about a 28x28 pixel image

In [ ]:
28*28

# Pixelised image as bitmap

In this dataset, the image of a digit is represented with 28x28 = 784 pixels.

Each pixel can go from black (255) to white (0). A similar approach can be used for a color picture using the the RGB code.

Matplotlib has a useful function to plot a bitmap image: [matplotlib.imshow](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.imshow.html)

Here is an example for a 3x3 bitmap image

In [ ]:
test_digit = np.array( [255,0,0,0,150,0,234 ,0,90])
test_image = test_digit.reshape(3,3)
print(test_image)
%matplotlib inline
plt.imshow(test_image, cmap = mpl.cm.binary,interpolation="nearest")

We now take a look at some of the digits in the data

In [ ]:
X, y = mnist["data"], mnist["target"]

#index = np.random.randint(0,70000)
index=36000
some_digit = X[index]
some_digit_image = some_digit.reshape(28, 28)


In [ ]:
plt.imshow(some_digit_image, cmap = mpl.cm.binary,
           interpolation="nearest")
plt.axis("off")

save_fig("some_digit_plot")
plt.show()

the target for this image is:

In [ ]:
print(y[index])

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
# EXTRA
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(9,9)) # 9 inches
example_images = np.r_[X[:12000:600], X[13000:30600:600], X[30600:60000:590]]
plot_digits(example_images, images_per_row=10)
save_fig("more_digits_plot")
plt.show()

we use the numpy `r_` function to concatenate objects

In [ ]:
np.r_?


# Training and test samples

- use first 60000 images for training 
- use the remaining 10000 for valdation and testing the algorithm 

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
plot_digit(X_train[55000])

In [ ]:
plot_digit(X_test[4000])

we first randomaise the training sample by generating randomaised index
 - this allows to have subsamples with random numbers
 - providing subsets witrgh just one digit are nor helpful for training
   - provide sets with both signal and background 

this is an example of generating ranom index of 6 objects

In [ ]:
for i in range(4):
    shuffle_index = np.random.permutation(6)
    print(shuffle_index)


now randomise the 60000 indices

In [ ]:
import numpy as np

shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

here we generate a random set of indices and change __both__ data (X) and its target (y).

**Reminder**: _data is a hand-written digit and target its true value_

# Binary classifier

In this first example we do somethign simple. Rather than identifiying all digits we want to train a binary classifer to identify number __5__

- positive/success: digit is __5__
- negative/fail: digit is __NOT 5__

In this example we use a _Stochastic Gradient Descent (SGD)_ classifier, provided in [SciKit-Learn](https://scikit-learn.org/stable/). 

Some of the feature of this classifier are
- efficient handling of large samples
- relatively fast to find the minimum of the cost function

For more details on SGD see for example these webpages [1](https://en.wikipedia.org/wiki/Stochastic_gradient_descent), [2](https://scikit-learn.org/stable/modules/sgd.html), [3](https://towardsdatascience.com/stochastic-gradient-descent-clearly-explained-53d239905d31)


The follwing boolean array will be used for training a classifier. They tell us which of the targets are actually a try hit (__5__).

In [ ]:
y_train[:50]

create boolean array to find put which digits are reall a `5`

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

In [ ]:
y_train_5[:50]

In [ ]:
plot_digit(X_train[2])

In [ ]:
plot_digit(X_train[13])

Here we import the `SGDClassifier` class from Scikit-Learn

**Note**: a few hyperparameters will have a different default value in future versions of Scikit-Learn, so a warning is issued if you do not set them explicitly. This is why we set `max_iter=5` and `tol=-np.infty`, to get the same results as in the book, while avoiding the warnings.

In order to provide reproducibility of the results we set the random seed. You can change it to something else and see if it has any effects. 

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=5, tol=-np.infty, random_state=42)
sgd_clf.fit(X_train, y_train_5)

- training is done with the `fit()` function
- we provide the `X_train` sample for training
- `y_train_5` is the boolean array telling the cliassifer the correct answer for each image. this is used by the algo to train and improve its performance

pick  some digit

In [ ]:
index=36123
some_digit = X[index]
print(some_digit)

In [ ]:
plot_digit(some_digit)

test the trained claissifer  on a few digits

In [ ]:
sgd_clf.predict([some_digit, X_train[2000], X_train[20]])

In [ ]:
plot_digit(X_train[2000])

In [ ]:
plot_digit(X_train[20])

# Measuring the performance of a classifier

A first natural perforemance measure could be the ratio of correct predictions called the __accuracy__.

We do this using the so-called __cross-validation__ provided by scikit-learn
- divide the training sample is sub samples of K elements (k-folds), for example triplets. There are 20000 triplets in this case. 
- train the classifier on 19999 tripltes and then measure its accuracy on the remaining  triplet. 
- repeat this for each of the 20000 triplets

There is a simple function for all these steps!

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

So it seems that the classifier has a 95% accuracy! 

But let's now compare to a quite stupid classifier that classifies all images as __not 5__

In [ ]:
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")

So even this clearly-wrong classifer has 90% accuracy!

why ?

Remember that 
- 90% of all images are __Not 5__
- 10% of them are a __5__

So by assigning __not 5__ to all images you can 90% of them right, which was the above accuracy.

So the conclusion is that __accuracy__ is not a good measure specially if the training sample is skewed as in this case

# Confusion Matrix

You should be already familiar with this concept from the statistics courses.

We have two classes of data: 
- class A or Signal
- class B or Background

and we define

- true positive (TP): signal classified as signal
- false positive (FP): background classified as signal
- false negative (FN): signal classified as background
- true negative (TN): background classified as background

These numbers compose what we call the __Confusion Matrix__. There is a function providing these numbers for the classifier

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

we first use the `cross_val_predict()` function and feed it with the training data and the correct answers (target).

This function performs k-fold validation , but instead of returning a score, it returns the precition as True or False for each test k-fold. 

Note that the fold being used for prediction was __not__ used for training. 

Then we compute the matrix

In [ ]:
print(y_train_5[:20])

In [ ]:
print(y_train_pred[:20])

In [ ]:
from sklearn.metrics import confusion_matrix

cm_pred  = confusion_matrix(y_train_5, y_train_pred)
cm_pred

Each row is the actual (true) class
- first row: true __Not 5__
- second row: true __5__

Each column is the predicted class
- first column: predicted __Not 5__
- second column: predicted __5__

the non-diagonal terms indicate the _false negative_ (signal classified as background) and _false positive_ (background classified as signal)

for a perfect classifier you would only have the diagonal terms

In [ ]:
y_train_perfect_predictions = y_train_5

In [ ]:
confusion_matrix(y_train_5, y_train_perfect_predictions)

# Precision and Recall

In practice, often other metrics are used to judge the goodness of the classifier for the purpose it was trained

__Precision__ is defined as the ratio __TP / (TP + FP)__
It tells use which fraction of the positives are true. So it is the __accuracy__ of the classifier

In [ ]:
3798 / (3798 + 710)

and of course there is function to compute it

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

Similarly, __Recall__ is defined as the ratio __TP / (TP + FN)__

It tells use which fraction of true positives are classified correctly. So it is also called the __True Positive Rate__ (TPR) and often referred to also as __sensitivity__

In [ ]:
3798 / (3798 + 1623)

which can also be obtained by calling a function

In [ ]:
recall_score(y_train_5, y_train_pred)

# Tradeoff between Precision and Recall

So after all our classifier is not so great: it has a precision of 84%, so out of the selected __5__'s, 84% of them are true. And it only detects ~70% of the __5__'s. 

Which is more important? higher precision or higher recall ?

The answer depends on the problem we want to solve.

In particle physics precision is often called  __purity__ and recall is called __efficiency__.

When doing b-quark flavor tagginng of jets you want to achieve good purity without loosing too much efficiency. 

Similar arguments apply to a trigger decision
- a trigger decision is used to store or throw away each of the collision events at a Collider
- you define an algorithm or a set of selection criteria to select events with a good chance of contaiing an event of interest, e.g. a Higgs boson
- you want high efficiency: out of 1000 true signal events you would like to retain as manny as possible
- you also want high purity for the sample: out of 1000 stored events, you want the fraction of signal events to be as large as possibile

Balancing these two requests is your task. Let's see an example with our `SGDClassifier`

### Performance curve for `SGDClassifier`

The decision making process for the  `SGDClassifier` is the following
1. compute a score for a data point (an image) using a __decision function__
1. score >= threshold:  data point assigned to positive class/signal/class A/True
1. score < threshold: data point assigned to negative class/background/class B/False
 
 The  __threshold__ is the key parameter to balance precision and recall
 1. high threshold: high precision(purity) but low recall (efficiency)
 1. low threshold: high recall (efficinecy) but low precision (purity)
 
 
 
 You cannot fix  the threshold with scikit-learn but you can do the following to study the performance and select your preferred __working point__ for the classifier
1. use the `decision_function()` instead of `predict()`
  - no binary decision but rather a numerical value
1. vary the threshold between a  minimum and a maximum
1. study precision and recall as a functio of threshold 
1. find the threshold value provisding the  (precision,recall) pair fitting your needs
1. use this thereshold for decision making



Let's compute the score for `some_digit`

In [ ]:
y_scores = sgd_clf.decision_function([some_digit])
y_scores

In [ ]:
threshold = 0
y_some_digit_pred = (y_scores > threshold)

In [ ]:
y_some_digit_pred

By default, the `SGDClassifier` uses threshold=0. So we are simply reproducing the behavior of the `predict()` function for the classifier.

However, since we now have access to the decision score, we can change the threshold and see how it affects the classifier prediction and its performance

In [ ]:
threshold = 200000
y_some_digit_pred = (y_scores > threshold)
y_some_digit_pred

as expected, for a high threshold the recall goes down our `some_digit` is not selected.

In order to choose the best threshold for our needs we do the following
1. compute the `decision_function()` for all instances in the trainning set
1. vary the threshold from a minimum to a maximum value
  1. for each threshold, compute precision and recall
  1. plot precision and recall as a function of the threshold
1. select the appropriate working point

This can be done easily in scikit-learn, again using the `cross_val_predict` and asking for the `decision_function` instead of the `prediction`

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                             method="decision_function")

In [ ]:
y_scores.shape

Instead of iterating over the thresholds, we use a function provided in scikit-learn.

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

In [ ]:
type(precisions)

In [ ]:
thresholds.shape

In [ ]:
precisions.shape

In [ ]:
recalls.shape

In [ ]:
thresholds[:10]

In [ ]:
thresholds[-10:]

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])
    plt.grid(all)

plt.figure(figsize=(10, 6))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.xlim([-800000, 800000])
save_fig("/tmp/precision_recall_vs_threshold_plot")
plt.show()

In [ ]:
y_scores.max()

In [ ]:
y_scores.min()

In [ ]:
(y_train_pred == (y_scores > 0)).all()

# Working with customised threshold

Now suppose we want to have a classifier witgh 80% precision. We can zoom in to find the threshold

In [ ]:
plt.figure(figsize=(10, 6))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.xlim([-100000, 300000])
plt.show()

which appears to happen for a threshold of 100000. So we can build our custom precition with a precision of 80% 

In [ ]:
y_train_pred_80 = (y_scores > 100000)

In [ ]:
precision_score(y_train_5, y_train_pred_80)

In [ ]:
recall_score(y_train_5, y_train_pred_80)

# Precision vs. recall 

It is often popular and useful to report the same information in a different plot. Rather than finding the threshold we  emphasise the working point in terms of precision and recall

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(all)

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)
save_fig("/tmp/precision_vs_recall_plot")
plt.show()

# ROC curves

There are other commonly used plots showing the performance of a classification algorithm. 

One of the most popular ones is the __receiver operating characteristic__ (ROC) curve, which became popular during World War II in the analysis of the radar signals. See some details on [wikipedia](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)


The ROC curve shows the __True Positive Rate__ as a function of the __False Positive Rate__. The ideal classifiers has the highest possible TPR and FPR ~ 0.

All these quantities are easily computed in scikit-learn

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
save_fig("/tmp/roc_curve_plot")
plt.show()

The diagonal is the performance of a random classifier. 

Best classifiers are as close as possible to the top left corner.

The ROC curve provides a quantitative comparison between different classifiers. 

Rather than just looking at the ROC curve, one can compute the __area under the curve__ (AUC). The classifier with the __largest AUC__ has the __best performace__.

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_5, y_scores)

# _SGD_ vs _Random Forrest_

To see the AUC at work,  we use  `RandomForestClassifier` to compute the ROC curve and the AUC and compare to the `SGDClassifier`.

**Note**: we set `n_estimators=20` to avoid a warning about the fact that its default value will be set to 100 in Scikit-Learn 0.22.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=20, random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")

There are some differences in using this new class
- `RandomForrestClassifier`  does not have a `decision_function()`. Instead it provides a predicted probablity function `predict_proba()`
  - all classifiers are requires to provide either `decision_function()` or `predict_proba()`
- `predict_proba()` returns an array of probabilities: one row for each instance (data point), one column for each class
  - column 0: probability of  __Not 5__
  - column 1: probability of being __5__
  
- to make the ROC curve we need a score for each instance
- we use the probaility of __being 5__ as the score to make the ROC curve

In [ ]:
y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

Finally we plot both the `SGD` and the `Random Forrest` ROC curves

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right", fontsize=16)
save_fig("/tmp/roc_curve_comparison_plot")
plt.show()

In [ ]:
roc_auc_score(y_train_5, y_scores_forest)

As expected `Random Forrest` has a might higher AUC! This is reflected also in the precision and recall values

In [ ]:
y_train_pred_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3)
precision_score(y_train_5, y_train_pred_forest)

In [ ]:
recall_score(y_train_5, y_train_pred_forest)

So now we have a precision (purity) of 99% and a recall (efficiency) of 86%.

This is to be compared to 80% (precision) and 72% (recall) for the `SGDClassifier`!

You might conclude that `Random Forrest` is __MUCH__ better than `SGD` ... 

... unfortunatly this is not correct.

The training sample has very few __5__'s (true positive) compared to __non 5__'s (true negative).

# Chossing between the ROC curve and the Precision/Recall (PR) plot

Typically PR plot is preferred __if__
- number of true positives (signal) is very small (e.g. rare signal)
- you care more about the false positives (fake signal) than false negative (efficiency loss)

otherwise use the ROC curve.

The PR plot for `Random Forest` is 

In [ ]:
precisions_forest, recalls_forest, thresholds_forest = precision_recall_curve(y_train_5, y_scores_forest)

In [ ]:
plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions_forest, recalls_forest)

And since we have few __5__'s compared to __Non 5__'s we better compare the PR plots.

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(precisions, recalls, "b:", linewidth=2, label="SGD")
plot_roc_curve(precisions_forest, recalls_forest, "Random Forest")
plt.legend(loc="lower left", fontsize=16)
plt.show()